<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="../../resources/logo.png" alt="Intellinum Bootcamp" style="width: 400px; height: 200px">
</div>

# Aggregations and JOINs
Apache Spark&trade; allow you to create on-the-fly data lakes.

## In this lesson you:
* Use basic aggregations.
* Correlate two data sets with a join.

In [1]:
#MODE = "LOCAL"
MODE = "CLUSTER"

import sys
from pyspark.sql import SparkSession
from pyspark import SparkConf
import os
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkConf
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.storagelevel import StorageLevel
from matplotlib import interactive
interactive(True)
import matplotlib.pyplot as plt
%matplotlib inline
import json
import math
import numbers
import numpy as np
import plotly
plotly.offline.init_notebook_mode(connected=True)

sys.path.insert(0,'../../src')
from settings import *

try:
    fh = open('../../libs/pyspark24_py36.zip', 'r')
except FileNotFoundError:
    !AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY} AWS_SECRET_ACCESS_KEY={AWS_SECRET_KEY} aws s3 cp s3://yuan.intellinum.co/bins/pyspark24_py36.zip ../../libs/pyspark24_py36.zip

try:
    spark.stop()
    print("Stopped a SparkSession")
except Exception as e:
    print("No existing SparkSession")

SPARK_DRIVER_MEMORY= "2G"
SPARK_DRIVER_CORE = "1"
SPARK_EXECUTOR_MEMORY= "1G"
SPARK_EXECUTOR_CORE = "1"
SPARK_EXECUTOR_INSTANCES = 6



conf = None
if MODE == "LOCAL":
    os.environ["PYSPARK_PYTHON"] = "/home/yuan/anaconda3/envs/pyspark24_py36/bin/python"
    conf = SparkConf().\
            setAppName("pyspark_day02_joins").\
            setMaster('local[*]').\
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars', '../../libs/mysql-connector-java-5.1.45-bin.jar').\
            set('spark.jars.packages','net.java.dev.jets3t:jets3t:0.9.0,com.google.guava:guava:16.0.1,com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1')
else:
    os.environ["PYSPARK_PYTHON"] = "./MN/pyspark24_py36/bin/python"
    conf = SparkConf().\
            setAppName("pyspark_day02_joins").\
            setMaster('yarn-client').\
            set('spark.executor.cores', SPARK_EXECUTOR_CORE).\
            set('spark.executor.memory', SPARK_EXECUTOR_MEMORY).\
            set('spark.driver.cores', SPARK_DRIVER_CORE).\
            set('spark.driver.memory', SPARK_DRIVER_MEMORY).\
            set("spark.executor.instances", SPARK_EXECUTOR_INSTANCES).\
            set('spark.sql.files.ignoreCorruptFiles', 'true').\
            set('spark.yarn.dist.archives', '../../libs/pyspark24_py36.zip#MN').\
            set('spark.sql.shuffle.partitions', '5000').\
            set('spark.default.parallelism', '5000').\
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars.packages','net.java.dev.jets3t:jets3t:0.9.0,com.google.guava:guava:16.0.1,com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1'). \
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars', 's3://yuan.intellinum.co/bins/mysql-connector-java-5.1.45-bin.jar')
        

spark = SparkSession.builder.\
    config(conf=conf).\
    getOrCreate()


sc = spark.sparkContext

sc.addPyFile('../../src/settings.py')

sc=spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.access.key", AWS_ACCESS_KEY)
hadoop_conf.set("fs.s3a.secret.key", AWS_SECRET_KEY)
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version", "2")

def display(df, limit=10):
    return df.limit(limit).toPandas()

def dfTest(id, expected, result):
    assert str(expected) == str(result), "{} does not equal expected {}".format(result, expected)

No existing SparkSession


## Basic Aggregations

Using [built-in Spark functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$), you can aggregate data in various ways. 

Run the cell below to compute the average of all salaries in the people DataFrame.

 By default, you get a floating point value.

In [2]:
peopleDF = spark.read.parquet("s3a://data.intellinum.co/bootcamp/common/people-10m.parquet")

In [3]:
from pyspark.sql.functions import avg
avgSalaryDF = peopleDF.select(avg("salary").alias("averageSalary"))

avgSalaryDF.show()

+-----------------+
|    averageSalary|
+-----------------+
|72633.01037020104|
+-----------------+



Convert that value to an integer using the `round()` function. See
[the documentation for round()](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$)
for more details.

In [4]:
from pyspark.sql.functions import round
roundedAvgSalaryDF = avgSalaryDF.select(round("averageSalary").alias("roundedAverageSalary"))

roundedAvgSalaryDF.show()

+--------------------+
|roundedAverageSalary|
+--------------------+
|             72633.0|
+--------------------+



In addition to the average salary, what are the maximum and minimum salaries?

In [5]:
from pyspark.sql.functions import min, max
salaryDF = peopleDF.select(max("salary").alias("max"), 
                           min("salary").alias("min"), 
                           round(avg("salary")).alias("averageSalary"))

salaryDF.show()

+------+------+-------------+
|   max|   min|averageSalary|
+------+------+-------------+
|180841|-26884|      72633.0|
+------+------+-------------+



## Joining Two Data Sets

Correlate the data in two data sets using a DataFrame join. 

The `people` data set has 10 million names in it. 

> How many of the first names appear in Social Security data files? 

To find out, use the Social Security data set with first name popularity data from the United States Social Security Administration. 

For every year from 1880 to 2014, `s3a://data.intellinum.co/bootcamp/common/names-1880-2016.parquet/` lists the first names of people born in that year, their gender, and the total number of people given that name. 

By joining the `people` data set with `names-1880-2016`, weed out the names that aren't represented in the Social Security data.

(In a real application, you might use a join like this to filter out bad data.)

In [6]:
ssaDF = spark.read.parquet("s3a://data.intellinum.co/bootcamp/common/names-1880-2016.parquet")

display(ssaDF)

,firstName,gender,total,year
0,Racine,F,5,1962
1,Clarisa,F,5,1946
2,Commodore,M,5,1897
3,Phil,M,264,1935
4,Paula,F,44,1891
5,Lenore,F,53,1979
6,Charlie,F,134,1940
7,Blanche,F,227,1955
8,Reynaldo,M,68,1927
9,Zack,M,19,1889


Next, with a quick count of distinct names, get an idea of how many distinct names there are in each of the tables.

DataFrames have a `distinct` method just for this purpose.

In [7]:
peopleDistinctNamesDF = peopleDF.select("firstName").distinct()

In [8]:
peopleDistinctNamesDF.count()

5113

In preparation for the join, let's rename the `firstName` column to `ssaFirstName` in the Social Security DataFrame.

Question to ponder: why would we want to do this?

Answer: In order that not confused by two firstNames in two data sets.

In [9]:
ssaDistinctNamesDF = ssaDF.select("firstName").withColumnRenamed("firstName",'ssaFirstName').distinct()

Count how many distinct names in the Social Security DataFrame.

In [10]:
ssaDistinctNamesDF.count()

96174

Now join the two DataFrames.

In [11]:
from pyspark.sql.functions import col
joinedDF = peopleDistinctNamesDF.join(ssaDistinctNamesDF, col("firstName") == col("ssaFirstName"))

How many are there?

In [12]:
joinedDF.count()

5096

## Exercise 1

In the tables above, some of the salaries in the `peopleDF` DataFrame are negative. 

These salaries represent bad data. 

Your job is to convert all the negative salaries to positive ones, and then sort the top 20 people by their salary.

**Hint:** See the Apache Spark documentation, [built-in Spark functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$).

### Step 1
Create a DataFrame`PeopleWithFixedSalariesDF`, where all the negative salaries have been converted to positive numbers.

In [13]:
# TODO

from pyspark.sql.functions import abs
peopleWithFixedSalariesDF = peopleDF.select(abs(col("salary")).alias("salary"))
display(peopleWithFixedSalariesDF)

,salary
0,74468
1,55688
2,92511
3,48451
4,90330
5,84103
6,54632
7,63966
8,97736
9,95509


In [14]:
# TEST - Run this cell to test your solution.

belowZero = peopleWithFixedSalariesDF.filter(peopleWithFixedSalariesDF["salary"] < 0).count()
dfTest("DF-L3-belowZero", 0, belowZero)

print("Tests passed!")

Tests passed!


### Step 2

Starting with the `peopleWithFixedSalariesDF` DataFrame, create another DataFrame called `PeopleWithFixedSalariesSortedDF` where:
0. The data set has been reduced to the first 20 records.
0. The records are sorted by the column `salary` in ascending order.

In [15]:
# TODO
from pyspark.sql.functions import asc
peopleWithFixedSalariesSortedDF = peopleWithFixedSalariesDF.orderBy(asc("salary")).limit(20)
display(peopleWithFixedSalariesSortedDF)

,salary
0,2
1,3
2,4
3,5
4,6
5,8
6,15
7,16
8,16
9,17


In [16]:
# TEST - Run this cell to test your solution.

resultsDF = peopleWithFixedSalariesSortedDF.select("salary")
dfTest("DF-L3-count", 20, resultsDF.count())

print("Tests passed!")

Tests passed!


In [17]:
# TEST - Run this cell to test your solution.

from pyspark.sql import Row

results = resultsDF.collect()

dfTest("DF-L3-fixedSalaries-0", Row(salary=2), results[0])
dfTest("DF-L3-fixedSalaries-1", Row(salary=3), results[1])
dfTest("DF-L3-fixedSalaries-2", Row(salary=4), results[2])

dfTest("DF-L3-fixedSalaries-10", Row(salary=19), results[10])
dfTest("DF-L3-fixedSalaries-11", Row(salary=19), results[11])
dfTest("DF-L3-fixedSalaries-12", Row(salary=20), results[12])

dfTest("DF-L3-fixedSalaries-17", Row(salary=28), results[17])
dfTest("DF-L3-fixedSalaries-18", Row(salary=30), results[18]) 
dfTest("DF-L3-fixedSalaries-19", Row(salary=31), results[19]) 

print("Tests passed!")

Tests passed!


## Exercise 2

As a refinement, assume all salaries under $20,000 represent bad rows and filter them out.

Additionally, categorize each person's salary into $10K groups.

### Step 1
 Starting with the `peopleWithFixedSalariesDF` DataFrame, create a DataFrame called `peopleWithFixedSalaries20KDF` where:
0. The data set excludes all records where salaries are below $20K.
0. The data set includes a new column called `salary10k`, that should be the salary in groups of 10,000. For example:
  * A salary of 23,000 should report a value of "2".
  * A salary of 57,400 should report a value of "6".
  * A salary of 1,231,375 should report a value of "123".

In [18]:
# TODO
peopleWithFixedSalaries20KDF = (peopleWithFixedSalariesDF
                                .filter(col("salary")>=20000)
                                .withColumn("salary10k", round(col("salary")/10000)))
display(peopleWithFixedSalaries20KDF)

,salary,salary10k
0,74468,7.0
1,55688,6.0
2,92511,9.0
3,48451,5.0
4,90330,9.0
5,84103,8.0
6,54632,5.0
7,63966,6.0
8,97736,10.0
9,95509,10.0


In [19]:
# TEST - Run this cell to test your solution.

below20K = peopleWithFixedSalaries20KDF.filter("salary < 20000").count()
 
dfTest("DF-L3-count-salaries", 0, below20K)  

print("Tests passed!")

Tests passed!


In [20]:
# TEST - Run this cell to test your solution.

from pyspark.sql.functions import count
results = (peopleWithFixedSalaries20KDF 
  .select("salary10k") 
  .groupBy("salary10k") 
  .agg(count("*").alias("total")) 
  .orderBy("salary10k") 
  .limit(5) 
  .collect()
)

dfTest("DF-L3-countSalaries-0", Row(salary10k=2.0, total=43792), results[0])
dfTest("DF-L3-countSalaries-1", Row(salary10k=3.0, total=212630), results[1])
dfTest("DF-L3-countSalaries-2", Row(salary10k=4.0, total=536535), results[2])
dfTest("DF-L3-countSalaries-3", Row(salary10k=5.0, total=1055261), results[3])
dfTest("DF-L3-countSalaries-4", Row(salary10k=6.0, total=1623248), results[4])

print("Tests passed!")

Tests passed!


## Exercise 3

Using the `peopleDF` DataFrame, count the number of females named Caren who were born before March 1980. 

### Step 1

Starting with `peopleDF`, create a DataFrame called `carensDF` where:
0. The result set has a single record.
0. The data set has a single column named `total`.
0. The result counts only 
  * Females (`gender`)
  * First Name is "Caren" (`firstName`)
  * Born before March 1980 (`birthDate`)

In [21]:
# TODO

carensDF = (peopleDF.filter("gender = 'F'")
            .filter("firstName='Caren'")
            .filter("birthDate < '1980-03'")
            .agg(count("*").alias("total")))

display(carensDF)

,total
0,750


In [22]:
# TEST - Run this cell to test your solution.

rows = carensDF.collect()

dfTest("DF-L3-carens-len", 1, len(rows))
dfTest("DF-L3-carens-total", Row(total=750), rows[0])

print("Tests passed!")

Tests passed!


## Review Questions
**Q:** What is the DataFrame equivalent of the SQL statement `SELECT count(*) AS total`  
**A:** ```.agg(count("*").alias("total"))```

**Q:** What is the DataFrame equivalent of the SQL statement 
```SELECT firstName FROM PeopleDistinctNames INNER JOIN SSADistinctNames ON firstName = ssaFirstName```  
**A:** 
`peopleDistinctNamesDF.join(ssaDistinctNamesDF, peopleDistinctNamesDF(col("firstName")) == col("ssaFirstName"))`

## Challenge Exercise 3

Starting with the `s3a://data.intellinum.co/bootcamp/common/names-1880-2016.parquet/` file, find the most popular first name for girls in 1885, 1915, 1945, 1975, and 2005.

### Step 1
Create a temporary view called `HistoricNames` where:
0. You may need to create temporary DataFrames to generate the DataFrame listing the names.
0. The result has three columns:
  * `firstName`
  * `year`
  * `total`

**Hint:** This is an example of a nested `SELECT` if you were to use SQL syntax. Using DataFrames, you will need to craft two queries and perform a `join`.

### Step 2

Load `s3a://data.intellinum.co/bootcamp/common/names-1880-2016.parquet/` into a DataFrame called `ssaDF` and display the schema.

In [23]:
# TODO

ssaDF = spark.read.parquet("s3a://data.intellinum.co/bootcamp/common/names-1880-2016.parquet/")

ssaDF.printSchema()

root
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- total: integer (nullable = true)
 |-- year: integer (nullable = true)



In [24]:
ssaDF.count()

1891894

### Step 3

Craft a DataFrame query that solves the problem described above.

Display the output of the DataFrame.

In [25]:
# TODO
# find the most popular first name for girls in 1885, 1915, 1945, 1975, and 2005.

from pyspark.sql.functions import col, max

maxNamesDF = (ssaDF.filter(col("year").isin('1885','1915','1945','1975','2005'))
              .filter("gender = 'F'")
              .groupBy("year")
              .agg(max("total").alias("total"))
              .withColumnRenamed("year","maxyear"))

outerQueryDF = maxNamesDF.join(ssaDF, "total","right_outer")

joinedQueryDF = (outerQueryDF.filter("maxyear >0")
                 .filter("maxyear == year")
                 .drop("maxyear")
                 .orderBy(asc("year")))

joinedQueryDF.show()

+-----+---------+------+----+
|total|firstName|gender|year|
+-----+---------+------+----+
| 9128|     Mary|     F|1885|
|58187|     Mary|     F|1915|
|59288|     Mary|     F|1945|
|58186| Jennifer|     F|1975|
|23934|    Emily|     F|2005|
+-----+---------+------+----+



In [26]:
# TEST - Run this cell to test your solution.

resultsDF = joinedQueryDF.select("firstName", "year", "total")
results = [ (r[0]+" "+str(r[1])+": "+str(r[2])) for r in resultsDF.collect()]

dfTest("DF-L3-Opt-historicNames-0", u'Mary 1885: 9128', results[0])
dfTest("DF-L3-Opt-historicNames-1", u'Mary 1915: 58187', results[1])
dfTest("DF-L3-Opt-historicNames-2", u'Mary 1945: 59288', results[2])
dfTest("DF-L3-Opt-historicNames-3", u'Jennifer 1975: 58186', results[3])
dfTest("DF-L3-Opt-historicNames-4", u'Emily 2005: 23934', results[4])
print("Tests passed!")

Tests passed!


&copy; 2019 [Intellinum Analytics, Inc](http://www.intellinum.co). All rights reserved.<br/>